<a href="https://colab.research.google.com/github/neerajthandayan/CourseProject/blob/main/Notebooks/Racist_Incidents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Calculating Racist Incident Rate Across England and Wales Police Force Regions

In [1]:
# Libraries

import pandas as pd
import numpy as np

In [2]:
# Data Retrieved From: https://www.gov.uk/government/statistics/racist-incidents-england-and-wales-2010-to-2011

!wget 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/116286/hosf0111-xls.xls'

--2021-04-27 02:21:16--  https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/116286/hosf0111-xls.xls
Resolving assets.publishing.service.gov.uk (assets.publishing.service.gov.uk)... 151.101.0.144, 151.101.64.144, 151.101.128.144, ...
Connecting to assets.publishing.service.gov.uk (assets.publishing.service.gov.uk)|151.101.0.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44544 (44K) [application/vnd.ms-excel]
Saving to: ‘hosf0111-xls.xls.2’

hosf0111-xls.xls.2  100%[===================>]  43.50K  --.-KB/s    in 0.01s   

2021-04-27 02:21:16 (3.99 MB/s) - ‘hosf0111-xls.xls.2’ saved [44544/44544]



In [3]:
hc = pd.read_excel('hosf0111-xls.xls', header=26)
hc = hc.iloc[:45,2:7].drop(columns=['Unnamed: 3','2008/09','2009/10'])
hc = hc.dropna()
hc.columns = ['Geography','Racist_Incidents']
hc.sort_values(by='Racist_Incidents', ascending=False).head()

,Geography,Racist_Incidents
44,England and Wales,51187.0
23,Metropolitan Police,9405.0
13,Greater Manchester,3288.0
38,West Yorkshire,2803.0
37,West Midlands,2646.0


#### Total Offences Data

In [4]:
# Total Offence Data Retrieved From: https://www.gov.uk/government/statistics/crimes-detected-in-england-and-wales-2010-to-2011

!wget 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/116406/hosb1111-pfatabs.xls'

--2021-04-27 02:21:16--  https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/116406/hosb1111-pfatabs.xls
Resolving assets.publishing.service.gov.uk (assets.publishing.service.gov.uk)... 151.101.0.144, 151.101.64.144, 151.101.128.144, ...
Connecting to assets.publishing.service.gov.uk (assets.publishing.service.gov.uk)|151.101.0.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99840 (98K) [application/vnd.ms-excel]
Saving to: ‘hosb1111-pfatabs.xls.2’

hosb1111-pfatabs.xl 100%[===================>]  97.50K  --.-KB/s    in 0.02s   

2021-04-27 02:21:17 (4.72 MB/s) - ‘hosb1111-pfatabs.xls.2’ saved [99840/99840]



In [5]:
off = pd.read_excel('hosb1111-pfatabs.xls', sheet_name='Table 1', header=6)
off = off.iloc[:63,:2].dropna().reset_index(drop=True)
off.columns = ['Geography','Total_Offences']
off.head()

,Geography,Total_Offences
0,Cleveland,43018.0
1,Durham,36517.0
2,Northumbria,82299.0
3,North East Region,161834.0
4,Cheshire,63729.0


#### Merging DataFrames to estimate Ethnic Hate Crime Rate

In [6]:
ecr = hc.merge(off, on='Geography')

# To calculate the racist incident rate, we take the ration bewteen racist incident and sum of racist incedents and total crimes commited.
# Here the sume of the RI and total crimes are taken as race indenst are not incorporated when calculation tital offences.
ecr['RI_Rate'] = ecr['Racist_Incidents']/(ecr['Total_Offences'] + ecr['Racist_Incidents'])
ecr.sort_values(by='RI_Rate', ascending=False).head(10)

,Geography,Racist_Incidents,Total_Offences,RI_Rate
40,South Wales,1974.0,93789.0,0.020613
26,South Yorkshire,2019.0,102741.0,0.019273
27,Staffordshire,1354.0,70890.0,0.018742
16,Leicestershire,1342.0,71355.0,0.018460
12,Hertfordshire,1133.0,66441.0,0.016767
15,Lancashire,1735.0,102503.0,0.016645
22,Northamptonshire,801.0,49288.0,0.015992
29,Surrey,1002.0,65125.0,0.015153
25,Nottinghamshire,1256.0,84263.0,0.014687
35,West Yorkshire,2803.0,189388.0,0.014584


In [7]:
ecr[['Geography','RI_Rate']].to_csv('racist_incidents.csv')